In [521]:
# Chase Abram
# using Pkg
# Pkg.add("DataFrames")
# using CSV
# using DataFrames
# # df = DataFrame()
# df = CSV.read("psetOne.csv", DataFrame)

# Pkg.add("ForwardDiff")
using ForwardDiff
using LinearAlgebra
using Optim

using Random
rng = MersenneTwister(12345)

MersenneTwister(UInt32[0x00003039], Random.DSFMT.DSFMT_state(Int32[-870096391, 1072918504, -1812426662, 1073255081, -733866021, 1073404543, 807620846, 1073368448, 1919433844, 1072852359  …  -362113007, 1073100625, -166402106, 1073460158, -1907020342, 721295190, -750225566, -1300227565, 382, 0]), [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], UInt128[0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000  …  0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x000000000

In [161]:
names(df)
df17 = df[df."Market" .== 17, :]
# df17."shares"
# X = convert(Matrix, df17[:, filter(x -> x in ["Constant", "EngineSize", "SportsBike", "Brand2", "Brand3"], names(df17))])
Br = convert(Matrix, df17[:, filter(x -> x in ["Brand2", "Brand3"], names(df17))])
Br = Bool.(Br)

7×2 BitArray{2}:
 0  0
 0  0
 1  0
 1  0
 0  1
 0  1
 0  1

In [582]:
function u_bar_jt(delta, sigma, X, zeta)
    return delta .+ X*sigma*zeta
end

function Prij(u_bar)
    return exp.(u_bar)./sum(exp.(u_bar))
end

# Shares
function s_logit(p, alpha, beta, X, xi)
    # Computer nmuerators for each s_j
    nums = exp.(X*beta .- alpha.*p[2:end] .+ xi)
    
    # return s_j, starting with s_0
    return [1/(1 + sum(nums)); nums./(1 + sum(nums))]
end

# Jacobian of shares
function s_logit_jac(p, alpha, beta, X, xi)
    # Get shares
    s = s_logit(p, alpha, beta, X, xi)
    
    # Jacobian is the outer product times alpha in this case...
    jac = alpha .* s * s'
    
    # ... except for an extra term on the diag
    for j in 1: length(p)
        jac[j,j] -= alpha*s[j]
    end
    
    return jac
end

# Hessian of shares (n by n by n tensor)
function s_logit_hess(p, alpha, beta, X, xi)
    
    # Initialize
    hess = zeros(length(p), length(p), length(p))
    
    # Fill in
    for j in 1:length(p)
        for k in 1:length(p)
            for l in 1:length(p)
                
                # Shares
                s = s_logit(p, alpha, beta, X, xi)
                
                # Jacobian of shares
                sj = s_logit_jac(p, alpha, beta, X, xi)
                
                if j == k # && j == l
                    # on diag of jacobian
                    hess[j,k,l] = alpha*sj[l,j]*(2*s[j] - 1)
                else
                    # off diag of jacobian
                    hess[j,k,l] = alpha*(sj[l,j]*s[k] + s[j]*sj[l,k])
                end
            end
        end
    end
    
    return hess
end

# Ownership structure
function Omega_star(br)
    
    # Initialize
    Om = zeros(size(br,1) + 1,size(br,1) + 1)
    
    for i in 1:size(Om,1) - 1
        for j in 1:size(Om,2) - 1
            if br[i,1] && br[j,1]
                # Brand2
                Om[i+1,j+1] = 1
            elseif br[i,2] && br[j,2]
                # Brand3
                Om[i+1,j+1] = 1
            elseif (!br[i,1]&& !br[j,1]) && (!br[i,2]&& !br[j,2])
                # Brand1
                Om[i+1,j+1] = 1
            end
        end
    end
    
    # Outside option
    Om[1,1] = 1
    
    return Om
end

# Negative of Omega in Nevo (2001)
function Omega(p, alpha, beta, X, xi, br)
    return Omega_star(br) .* s_logit_jac(p, alpha, beta, X, xi)
end

# Fixed point expression (want zero at FP)
function fp_logit(p, alpha, beta, X, xi, br)
    return s_logit(p, alpha, beta, X, xi) + Omega(p, alpha, beta, X, xi, br)*p
end

# (Jacobian of Omega)*p (this is used in the gradient of the FP expression)
function jac_om_p(p, alpha, beta, X, xi, br)
    jacomp = zeros(length(p), length(p))
    hess = s_logit_hess(p, alpha, beta, X, xi)
    oms = Omega_star(br)
    for i in 1:length(p)
        for j in 1:length(p)
            jacomp[i,j] = ((hess[i,:,j].* oms[i,j])'* p)[1]
        end
    end
    return jacomp
end

function fp_logit_2(p, alpha, beta, X, xi, br)
    return (fp_logit(p, alpha, beta, X, xi, br)' * fp_logit(p, alpha, beta, X, xi, br))[1]
end


# function fp_logit_2_jac(storage, p, alpha, beta, X, xi, br)
#     sp = s_logit(p, alpha, beta, X, xi)
#     Om = Omega_star(br).* s_logit_jac(p, alpha, beta, X, xi)
#     s_jac = s_logit_jac(p, alpha, beta, X, xi)
#     jacomp = jac_om_p(p, alpha, beta, X, xi, br)
    
#     jac = 2*(sp + Om*p)' * (s_jac + jacomp + Om)
#     for i in 1:length(p)
#         storage[i] = jac[i]
#     end
# #     storage = (sp + Om*p)' * (s_jac + jacomp + Om)
# #     storage = 0
# #     return storage
    
# end

function fp_logit_2_jac(storage, p, alpha, beta, X, xi, br)
    s = s_logit(p, alpha, beta, X, xi)
    oms = Omega_star(br)
    sj = s_logit_jac(p, alpha, beta, X, xi)
    sh = s_logit_hess(p, alpha, beta, X, xi)
    
    for k in 1:length(p)
        storage[k] = (2 .* (s + (oms .* sj)*p)' * (sj[k,:] + (oms .* sh[:,:,k])*p + sj[:,k]))[1]
    end
    return storage
end


function solve_fp_logit(alpha, beta, X, xi, br, tol = 1e-14, maxit = 100)
    
    f(x) = fp_logit_2(x, alpha, beta, X, xi, br)[1,1]
    g!(storage, x) = fp_logit_2_jac(storage, x, alpha, beta, X, xi, br)
    
    opt = optimize(f, p, NelderMead(), Optim.Options(f_tol = tol))
#     opt = optimize(f, g!, p, BFGS(), Optim.Options(time_limit = 5.0))
    return opt, opt.minimizer, opt.minimum
#     return opt
end



solve_fp_logit (generic function with 3 methods)

In [583]:
p = ones(size(X,1)+1,1)
p[7] = 1
alpha = 3
beta = [1 1 2 -1 1]'

# seed maintains results across runs
rng = MersenneTwister(1234)
xi = randn(rng, size(df17,1))

s_logit(p, alpha, beta, X, xi)
println("sum shares: ", sum(s_logit(p, alpha, beta, X, xi)))

s_logit_jac(p, alpha, beta, X, xi)

s_logit_hess(p, alpha, beta, X, xi)

# Omega_star(Br)

# Omega(p, alpha, beta, X, xi, Br)

# fp_logit(p, alpha, beta, X, xi, Br)' * fp_logit(p, alpha, beta, X, xi, Br)

# fp_logit_2(p, alpha, beta, X, xi, Br)
st = zeros(length(p))
fp_logit_2_jac(st, p, alpha, beta, X, xi, Br)
println(st)

# jac_om_p(p, alpha, beta, X, xi, Br)

# fp_logit_jac(p, alpha, beta, X, xi, Br)


op, opam, opm = solve_fp_logit(alpha, beta, X, xi, Br)
println("opam: ", opam)
println("test on fp: ", fp_logit(opam, alpha, beta, X, xi, Br))

# op = optimize(x -> (fp_logit(x, alpha, beta, X, xi, Br)' * fp_logit(x, alpha, beta, X, xi, Br))[1,1], p, LBFGS())

# pmin = op.minimizer
# println("p min: ", pmin)
# println("min: ", op.minimum)
# println("fp_logit: ", fp_logit(pmin, alpha, beta, X, xi, Br))
# df17[3, "Brand2"]

sum shares: 1.0
[5.0694701729343456e-5, 0.0024239009659556884, 0.0030534676272433864, 0.00022845517442355136, 0.0011220259776003948, 0.011759383413529603, -0.29120987877836574, 0.27257195091788344]
opam: [3.970021275086938; 0.37958236173424115; 0.37994905573014914; 0.32741313950161793; 2.7116367297024384; 2.7553053115731347; 2.5792069606547856; 2.5799608022555303]
test on fp: [-0.0019329810601636682; 8.076164753113152e-5; 2.6185127166580835e-5; 0.00014324649069367718; -0.000164621365818419; -0.0002808300810518432; -1.7153535733394065e-5; -4.8785723733145436e-5]


In [430]:
# NM: opam: [3.970021275086938; 0.37958236173424115; 0.37994905573014914; 
#     0.32741313950161793; 2.7116367297024384; 2.7553053115731347; 2.5792069606547856; 
#     2.5799608022555303]



function sHat(delta, X, sigma, zeta)
    
    
    # Compute u_bar_ijt
    
    # Compute prob.
    
    # Do above I times, then integrate
    
    
    return 0
end



sHat (generic function with 1 method)

In [456]:
function tester()
    f1(x, y, z) = (x[1] - 5 - y)^2 + (x[2] - 3 - z)^2
    
    function g(storage, x, y, z)
        storage[1] = 2*(x[1] - 5 - y)
        storage[2] = 2*(x[2] - 3 - z)
    end
    
    a = 40
    b = 5
    f(x) = f1(x,a,b)
    g!(storage, x) = g(storage, x, a, b)
    
    
    opt = optimize(f, g!, [0.0, 0.0], LBFGS())
    return opt, opt.minimizer, opt.minimum
end

tester()

( * Status: success

 * Candidate solution
    Final objective value:     0.000000e+00

 * Found with
    Algorithm:     L-BFGS

 * Convergence measures
    |x - x'|               = 4.50e+01 ≰ 0.0e+00
    |x - x'|/|x'|          = 1.00e+00 ≰ 0.0e+00
    |f(x) - f(x')|         = 2.09e+03 ≰ 0.0e+00
    |f(x) - f(x')|/|f(x')| = Inf ≰ 0.0e+00
    |g(x)|                 = 0.00e+00 ≤ 1.0e-08

 * Work counters
    Seconds run:   0  (vs limit Inf)
    Iterations:    1
    f(x) calls:    3
    ∇f(x) calls:   3
, [45.0, 8.0], 0.0)

In [548]:
p = 0.5*ones(size(X,1) + 1,1)
fp(x) = fp_logit_2(x, alpha, beta, X, xi, Br)
println(fp_logit_2_jac(st, p, alpha, beta, X, xi, Br))

ForwardDiff.gradient(fp, p)

[3.9929727590596066e-5, 0.011822701069386479, 0.2066532414565205, 0.00414575194000096, 0.015471991579947249, 0.016449738377790225, -0.6052484173047031, 0.350665063153467]


8×1 Array{Float64,2}:
  5.1405017651023615e-11
  0.008373081149785726
  0.14654326150353097
  0.0029362656998745892
  0.010958999382011163
  0.016453668029284087
 -0.6045129417687891
  0.3511811100518005